In [7]:
import sys
import time

class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile
    def write(self, message):
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write(message)  
            self.log.close()
sys.stdout = Logger(logfilename='logfile')

In [1]:
import pandas as pd
import os
import gc

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_pickle(PATH + 'inter/train_only_0.pkl')
test = pd.read_pickle(PATH + 'inter/test_only_0.pkl')

train.shape, test.shape

((307511, 329), (48744, 328))

In [2]:
import re
do_deleting = True
ptn = 'std'
for x in os.listdir(PATH + 'inter/'):
    if 'curr.pkl' in x:
        print(x)
        tmp = pd.read_pickle(PATH + 'inter/' + x)
        print(tmp.shape)
        if do_deleting:
            col = [i for i in tmp.columns if not re.findall(ptn, i)]
        else:
            col = [i for i in tmp.columns]
        train = train.merge(tmp[col], on='SK_ID_CURR', how='left')
        test = test.merge(tmp[col], on='SK_ID_CURR', how='left')
        print(train.shape, test.shape)
        del tmp
        gc.collect()
        

for x in os.listdir(PATH + 'inter/linear/'):
    if x.split('.')[-1] == 'pkl':
        if x != 'linearinstal2curr.pkl':
            print(x)
            tmp = pd.read_pickle(PATH + 'inter/linear/' + x)
            if do_deleting:
                col = [i for i in tmp.columns if not re.findall(ptn, i)]
            else:
                col = [i for i in tmp.columns]
            train = train.merge(tmp[col], on='SK_ID_CURR', how='left')
            test = test.merge(tmp[col], on='SK_ID_CURR', how='left')
            del tmp
            gc.collect()
            print(train.shape, test.shape)'
             
'done'

poscash2curr.pkl
(356255, 116)
(307511, 429) (48744, 428)
bureau2curr.pkl
(356255, 554)
(307511, 888) (48744, 887)
prev2curr.pkl
(356255, 610)
(307511, 1423) (48744, 1422)
credit2curr.pkl
(356255, 502)
(307511, 1804) (48744, 1803)
install2curr.pkl
(356255, 60)
(307511, 1854) (48744, 1853)
linearposcash2curr.pkl
(307511, 1858) (48744, 1857)
linearbureau2curr.pkl
(307511, 1874) (48744, 1873)
newlinearinst2curr.pkl
(307511, 1902) (48744, 1901)
linearcredit2curr.pkl
(307511, 1916) (48744, 1915)


'done'

In [3]:
train.to_pickle(PATH + 'haoyan_train.pkl')
test.to_pickle(PATH + 'haoyan_test.pkl')
train.shape

(307511, 1916)

In [1]:
import os
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
PATH = '/home/kai/data/kaggle/homecredit/'

In [2]:
# Read train and test
train_df = pd.read_pickle(PATH + 'inter/train_base.pkl')
test_df = pd.read_pickle(PATH + 'inter/test_base.pkl')
train_df.shape, test_df.shape

((307511, 2374), (48744, 2373))

In [4]:
cat_feat = [
    'NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE',
    'NAME_INCOME_TYPE','NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
    'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE',
    'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
    'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
    'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
    'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
    'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21','WEEKDAY_APPR_PROCESS_START', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 
    'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'
]
cat_feat = [x for x in train_df.columns if x in cat_feat]

In [ ]:
n_splits = 5
print('begin cv')
target = train_df['TARGET']
ignore_cols = ['ORGANIZATION_TYPE', 'TARGET', 'SK_ID_CURR']
features = [x for x in train_df.columns if x not in ignore_cols]
train = train_df[features]
test = test_df[features]

from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

lgbm_train = lgbm.Dataset(data=train,
                          label=target,
                          categorical_feature=[],
                          free_raw_data=False)

lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_iteration': 4000,
    'num_threads': 8,

    'num_leaves': int(round(44.368535336628419)),
    'feature_fraction': 0.28231763168020257,
    'bagging_fraction': 0.94901525271474951,
    'max_depth': int(round(8.0430115561596267)),
    'lambda_l1': 0.30680079516647751,
    'lambda_l2': 0.079128660903201031,
    'min_split_gain': 0.054005067457890979,
    'min_child_weight': 98.172643147364937
}

cv_results = lgbm.cv(train_set=lgbm_train,
                     params=lgbm_params,
                     nfold=n_splits,
                     seed=2018,
                     early_stopping_rounds=150,
                     verbose_eval=100,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

begin cv


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	cv_agg's auc: 0.779911 + 0.00411083
[200]	cv_agg's auc: 0.789804 + 0.00344758
[300]	cv_agg's auc: 0.792555 + 0.00336656
[400]	cv_agg's auc: 0.79362 + 0.00334096
[500]	cv_agg's auc: 0.793922 + 0.00331341


In [ ]:
clf = lgbm.train(train_set=lgbm_train,
                 params=lgbm_params,
                 num_boost_round=optimum_boost_rounds)

In [ ]:
y_pred = clf.predict(test)
out_df = pd.DataFrame({'SK_ID_CURR': test_df['SK_ID_CURR'], 'TARGET': y_pred})
out_df.to_csv(PATH+'/submission/haoyan_pred1_0629.csv', index = False)

In [ ]:
# new install linear
[100]	cv_agg's auc: 0.779285 + 0.00427557
[200]	cv_agg's auc: 0.789532 + 0.00380641
[300]	cv_agg's auc: 0.792621 + 0.00363398
[400]	cv_agg's auc: 0.793798 + 0.00379608
[500]	cv_agg's auc: 0.794238 + 0.0039459
[600]	cv_agg's auc: 0.794262 + 0.00387596
[700]	cv_agg's auc: 0.794274 + 0.00381348
Optimum boost rounds = 636
Best CV result = 0.7943991998600779

In [ ]:
# with linear with outliers
[100]	cv_agg's auc: 0.779877 + 0.00414474
[200]	cv_agg's auc: 0.789619 + 0.00362956
[300]	cv_agg's auc: 0.792776 + 0.00336169
[400]	cv_agg's auc: 0.793822 + 0.00331481
[500]	cv_agg's auc: 0.794346 + 0.00335585
[600]	cv_agg's auc: 0.794679 + 0.00340904
[700]	cv_agg's auc: 0.794795 + 0.00338389
[800]	cv_agg's auc: 0.794702 + 0.00342039
Optimum boost rounds = 725
Best CV result = 0.7948496327582341 

In [ ]:
# without linear with outliers
[100]	cv_agg's auc: 0.778547 + 0.00412467
[200]	cv_agg's auc: 0.789063 + 0.00344099
[300]	cv_agg's auc: 0.792374 + 0.00341128
[400]	cv_agg's auc: 0.793485 + 0.00322214
[500]	cv_agg's auc: 0.793913 + 0.00320875
[600]	cv_agg's auc: 0.793994 + 0.00324946
[700]	cv_agg's auc: 0.794062 + 0.0033091
[800]	cv_agg's auc: 0.793972 + 0.00308594
Optimum boost rounds = 701
Best CV result = 0.7940757965150353 

ALL
[100]	cv_agg's auc: 0.7795 + 0.00411813
[200]	cv_agg's auc: 0.789489 + 0.0034798
[300]	cv_agg's auc: 0.792526 + 0.00351702
[400]	cv_agg's auc: 0.7936 + 0.00338604
[500]	cv_agg's auc: 0.794217 + 0.00323238
[600]	cv_agg's auc: 0.794211 + 0.00315142
[700]	cv_agg's auc: 0.794229 + 0.00328479
Optimum boost rounds = 550
Best CV result = 0.7943004497519002


with std
[100]	cv_agg's auc: 0.779445 + 0.00383935
[200]	cv_agg's auc: 0.789236 + 0.00355325


No idea
[100]	cv_agg's auc: 0.779646 + 0.00419223
[200]	cv_agg's auc: 0.789602 + 0.00344841
[300]	cv_agg's auc: 0.792667 + 0.00343178
[400]	cv_agg's auc: 0.793703 + 0.00339853
[500]	cv_agg's auc: 0.794175 + 0.00342015

sth2prev2curr no sum
[100]	cv_agg's auc: 0.777611 + 0.00437966
[200]	cv_agg's auc: 0.784903 + 0.00415943
[300]	cv_agg's auc: 0.786765 + 0.00370779
[400]	cv_agg's auc: 0.787215 + 0.00346352
[500]	cv_agg's auc: 0.787577 + 0.00352841 